In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.io import read_image
import numpy as np
from torch.utils.data import DataLoader
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models import resnet18, ResNet18_Weights
import torch.nn as nn
import load_dataset
import common_functions
from time import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import unet_model_v2
import torch.nn.functional as F
from pytorch_msssim import MS_SSIM
from skimage import io

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
gaze_model_efficientnet = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=True).to(device)
gaze_model_efficientnet.classifier.fc = nn.Linear(1280, 2).to(device)
gaze_model_efficientnet.load_state_dict(torch.load("/data/volume_2/GazePrivacyModels/PretrainEffiNetXGaze/model.pt"))

gaze_model_efficientnet.eval()

gaze_estimator = resnet18().to(device)
gaze_estimator.fc = nn.Linear(512, 2).to(device)
gaze_estimator.load_state_dict(torch.load("/data/volume_2/GazePrivacyModelsV2/PretrainRes18GazeCapture/model.pt"))
gaze_estimator.eval()

loss_gaze = nn.L1Loss()
loss_list = []
GazeCapture_data = load_dataset.GazeCapture()
train_dataloader = DataLoader(GazeCapture_data, batch_size=100, shuffle=True)
image_array = np.zeros((500, 224, 224, 3))
loss_array = np.zeros((500))

In [ ]:
for batch, (X, y) in tqdm(enumerate(train_dataloader)):
    X, y = X.to(device), y[:, 0:2].to(device)
    gaze_pre = gaze_model_efficientnet(X)
    gaze_ori = gaze_estimator(X)
    loss = common_functions.angle_error_instance(gaze_pre,gaze_ori)
    X_array = X.cpu().data.numpy()
    X_array = np.transpose(X_array, (0, 2, 3, 1))
    image_array[batch*100:(batch+1)*100,:,:,:] = X_array
    loss = loss.cpu().data.numpy()
    loss_array[batch*100:(batch+1)*100]  = loss
    if batch == 4:
        break

In [ ]:
sort_index = np.argsort(loss_array)

In [ ]:
mean_face = np.zeros((224,224, 3))
min_loss = 100
for select_num in range(15, 50):
    mean_face = np.zeros((224,224, 3))
    for i in sort_index[0:select_num]:
        mean_face += image_array[i]

    mean_face /= select_num
    mean_face *= 255
    mean_face = mean_face.astype(np.uint8)
    io.imsave("/data/volume_2/Gaze_privacy_v2/Gaze_privacy/gazecapture_average_face_efficientnet_M700.png", mean_face)
    anchor_image = torch.unsqueeze(read_image("/data/volume_2/Gaze_privacy_v2/Gaze_privacy/gazecapture_average_face_efficientnet_M700.png")/255., 0).to(device)
    gaze_pre = gaze_model_efficientnet(anchor_image)
    gaze_ori = gaze_estimator(anchor_image)
    ang_loss = common_functions.avg_angle_error(gaze_pre, gaze_ori).item()
    if ang_loss<min_loss:
        min_loss = ang_loss
        fin_num = select_num
plt.imshow(mean_face)
plt.show()

In [ ]:
mean_face = np.zeros((224,224, 3))
for i in sort_index[0:fin_num]:
    mean_face += image_array[i]

mean_face /= fin_num
mean_face *= 255
mean_face = mean_face.astype(np.uint8)
io.imsave("/data/volume_2/Gaze_privacy_v2/Gaze_privacy/gazecapture_average_face_efficientnet_M700.png", mean_face)
anchor_image = torch.unsqueeze(read_image("/data/volume_2/Gaze_privacy_v2/Gaze_privacy/gazecapture_average_face_efficientnet_M700.png")/255., 0).to(device)
gaze_pre = gaze_model_efficientnet(anchor_image)
gaze_ori = gaze_estimator(anchor_image)
ang_loss = common_functions.avg_angle_error(gaze_pre, gaze_ori).item()
print(ang_loss)
plt.imshow(mean_face)
plt.show()